# Segmentation (bcg)

This file simplifies the process of the bcg customer segmentation. It analyzes customers on a matrix of frequency of purchase and average purchase price.

We import the last three years of classics and pops ticketing transactions. Segmentation is done separately for classics and pops due to differentiations in pricing and available concerts. This will allow us to determine those who are attendees of both or just one or the other.

In [149]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Imports

In [150]:
import os
import math
import pandas as pd
import numpy as np
import datetime as dt

## Inputs

In [151]:
fy_start = 16
fy_end = 19

## Data Import

In [152]:
def ticket_data_import(series, fy_start=10, fy_end=19, path='data/'):
    """
    This function retrieves all of the subscription files across a\
 user determined date range and series. 
    
    Keyword arguments:
    series -- either 'clx' or 'pops'
    fy_start -- earliest fiscal year in question (default 10)
    fy_end -- last fiscal year in question (default 19)
    path -- path to data (default 'data/')
    
    Returns:
    df -- pandas dataframe consisting of ticketing data with a fiscal year added to end
    """
    
    accepted_series = ['clx', 'pops']
    
    if series.lower() not in accepted_series:
        raise ValueError('series must be of accepted series types: ', 
                         accepted_series)
    
    dataframes = []
    for i in range(fy_start, fy_end+1):
        file = f"{series.capitalize()}{i}.csv"
        tmp = pd.read_csv(path + file, skiprows=3)
        tmp['fy'] = i
        dataframes.append(tmp)

    df = pd.concat(dataframes, ignore_index=True)
    
    return df

In [153]:
def date_conv(s):
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

Import the data, add a column to identify series, concatenate to one dataframe

In [339]:
clx_df = ticket_data_import('clx', fy_start, fy_end)
clx_df['series'] = 'clx'

pop_df = ticket_data_import('pops', fy_start, fy_end)
pop_df['series'] = 'pops'

tix_df = pd.concat([clx_df, pop_df])

Convert dates

In [340]:
tix_df['perf_dt'] = date_conv(tix_df['perf_dt'])

## Exclude Bad IDs

There are many customer IDs that should be excluded because they are corporations or internal employees or other reasons.

In [341]:
bad_ids = [
    0,                 # Unknown IDs
    2700674,           # Terry Dwyer
    955085,            # PSO Comps
    3141490,           # Symphony Shop
    91013,             # PSO Orchestra Members
    118401,            # PSO Prez - JF
    91006,             # PSO Artist Comps
    3328612,           # Development Guest
    925728,            # Kurt Mortenson (Internal)
    2010347,           # Goldstar
    2437127,           # Lorraine Caukin (Internal)
    2515897,           # Gregory Pierre Cox (internal)
    3080718,           # Gary Good
    91015,             # PSO Press
    120696             # Carl St. Clair
]

In [342]:
tix_df = tix_df[~tix_df['summary_cust_id'].isin(bad_ids)]

## Drop NaN

In [343]:
tix_df = tix_df[pd.notnull(tix_df['summary_cust_id'])].reset_index(drop=True)

## Remove Outliers

There are outliers from programs and internal purchases allowing prices below 5. There are also a few entries that are greater than 500 per a single ticket which must be an entry error. Exclude those from this.

In [344]:
paid_constraint_min = 5
paid_constraint_max = 500

In [345]:
tix_df = tix_df.loc[tix_df.paid_amt > paid_constraint_min]
tix_df = tix_df.loc[tix_df.paid_amt < paid_constraint_max]

## Determine Baselines

Ticket Prices dataframes

In [346]:
clx_price_df = tix_df.loc[(tix_df.series == 'clx')][['summary_cust_id', 'paid_amt']]\
                     .groupby(['summary_cust_id']).mean().reset_index()
pop_price_df = tix_df.loc[(tix_df.series == 'pops')][['summary_cust_id', 'paid_amt']]\
                     .groupby(['summary_cust_id']).mean().reset_index()

Frequency of Concerts dataframes

In [347]:
clx_freq_df = tix_df.loc[tix_df.series == 'clx'][['summary_cust_id', 'perf_dt']]\
                    .drop_duplicates().groupby(['summary_cust_id']).count().reset_index()

pop_freq_df = tix_df.loc[tix_df.series == 'pops'][['summary_cust_id', 'perf_dt']]\
                    .drop_duplicates().groupby(['summary_cust_id']).count().reset_index()

#### Classics Baselines

In [348]:
clx_baselines = {
    'avg_price': clx_price_df.paid_amt.mean(),
    'max_price': clx_price_df.paid_amt.max(),
    'min_price': clx_price_df.paid_amt.min(),
    'upper_price': None,
    'lower_price': None,
    'avg_freq': clx_freq_df.perf_dt.mean(),
    'max_freq': clx_freq_df.perf_dt.max(),
    'min_freq': clx_freq_df.perf_dt.min(),
    'upper_freq': None,
    'lower_freq': None
}

clx_baselines['upper_price'] = np.mean([clx_baselines['avg_price'], clx_baselines['max_price']])
clx_baselines['lower_price'] = np.mean([clx_baselines['avg_price'], clx_baselines['min_price']])
clx_baselines['upper_freq'] = np.mean([clx_baselines['avg_freq'], clx_baselines['max_freq']])
clx_baselines['lower_freq'] = np.mean([clx_baselines['avg_freq'], clx_baselines['min_freq']])

#### Pops Baselines

In [349]:
pop_baselines = {
    'avg_price': pop_price_df.paid_amt.mean(),
    'max_price': pop_price_df.paid_amt.max(),
    'min_price': pop_price_df.paid_amt.min(),
    'upper_price': None,
    'lower_price': None,
    'avg_freq': pop_freq_df.perf_dt.mean(),
    'max_freq': pop_freq_df.perf_dt.max(),
    'min_freq': pop_freq_df.perf_dt.min(),
    'upper_freq': None,
    'lower_freq': None
}

pop_baselines['upper_price'] = np.mean([pop_baselines['avg_price'], pop_baselines['max_price']])
pop_baselines['lower_price'] = np.mean([pop_baselines['avg_price'], pop_baselines['min_price']])
pop_baselines['upper_freq'] = np.mean([pop_baselines['avg_freq'], pop_baselines['max_freq']])
pop_baselines['lower_freq'] = np.mean([pop_baselines['avg_freq'], pop_baselines['min_freq']])

# Identifying renew/return/new

In [350]:
cust_current = tix_df.loc[tix_df.fy == fy_end]['summary_cust_id']\
                     .drop_duplicates().reset_index(drop=True)

In [351]:
cust_renew_range = tix_df.loc[tix_df.fy == (fy_end - 1)]['summary_cust_id']\
                         .drop_duplicates().reset_index(drop=True)

In [352]:
cust_return_range = tix_df.loc[(tix_df.fy == (fy_start + 1)) | (tix_df.fy == fy_start)]\
                          ['summary_cust_id'].drop_duplicates().reset_index(drop=True)

In [353]:
cust_recency = {}

for cust in cust_current:
    if cust in set(cust_renew_range):
        cust_recency.update({cust: 'renew'})
    elif cust in set(cust_return_range):
        cust_recency.update({cust: 'return'})
    else:
        cust_recency.update({cust: 'new'})

## Identifying segmentation

In [354]:
def segmentation(df, series, baselines, current_year=fy_end):
    """
    This function takes a price df and a frequency df and determines\
 the customer segmentation based on the baselines provided.
    
    Segments:
    aficionado -- above average ticket price, above upper bound frequency.
    high-value regular -- above upper bound ticket price, above lower bound frequency,\
 below upper bound frequency & above avg price, above avg frequency, below lower bound\
 frequency.
    committed low-budget -- below average ticket price, above average frequency.
    evolving concert-goer -- below upper bound ticket price, above lower bound frequency,\
 below avg frequency.
    high-value prospect -- above avg ticket price, below lower bound frequency.
    one-timer -- below avg ticket price, below lower bound frequency.
    
    
    Keyword arguments:
    df -- dataframe consisting of 'summary_cust_id' and 'paid_amt' ('paid_amt'\
 represents the avg price paid per ticket per customer.)
    series -- series must either be 'clx' or 'pops'
    baselines -- a dictionary containing: 
        {
            avg_price: 
            max_price:
            min_price:
            upper_price:
            lower_price:
            avg_freq:
            max_freq:
            min_freq:
            upper_freq:
            lower_freq:
        }
    current_year -- last two digits of fiscal year in question ex. 19 for fy19 (default fy_end)
    
    Returns:
    d -- dictionary taking 'summary_cust_id' as the key and a list of three items\
 as the value. The first item is the avg ticket price for the customer, the second\
 is the avg frequency for the customer, the third is the segment for the customer.
    """
    # Check if series is clx or pops:
    if series.lower() not in ['clx', 'pops']:
        raise Exception(f"series must be 'clx' or 'pops' not {series}")
    
    
    # Create dataframes for calculations
    df = df.loc[(df['series'] == series)]
    price_df = df[['summary_cust_id', 'paid_amt']].groupby(['summary_cust_id'])\
                                                  .mean().reset_index()
    freq_df = df[['summary_cust_id', 'perf_dt']].drop_duplicates().groupby(['summary_cust_id'])\
                                                .count().reset_index()
    current_cust = df.loc[df['fy'] == current_year]['summary_cust_id']
    
    # Initialize dataframes with unique customers from current year
    d = {cust: [0, 0, None] for cust in current_cust}
    
    # Loop all customers to look up their avg ticket price and frequency
    for k,v in d.items():
        v[0] = price_df.loc[price_df['summary_cust_id'] == k]['paid_amt'].values[0]
        v[1] = freq_df.loc[freq_df['summary_cust_id'] == k]['perf_dt'].values[0]

        # Test to ensure that values are all greater than 0, if not there is a problem:
        if (v[0] < 1) | (v[1] < 1):
            raise Exception("Either avg tix price or frequency is less than 1, that's bad.")
            
        # Fill in segments
        if (v[0] > baselines['avg_price']) & (v[1] > baselines['upper_freq']):
            d[k][2] = 'aficionado'
        elif ((v[0] > baselines['upper_price']) & (v[1] < baselines['upper_freq']) & (v[1] > baselines['lower_freq'])) |\
            ((v[0] > baselines['avg_price']) & (v[0] < baselines['upper_price']) & (v[1] < baselines['upper_freq']) & (v[1] > baselines['avg_freq'])):
            d[k][2] = 'high-value_regular'
        elif (v[0] < baselines['avg_price']) & (v[1] > baselines['avg_freq']):
            d[k][2] = 'committed_low-budget'
        elif (v[0] < baselines['upper_price']) & (v[1] < baselines['avg_freq']) & (v[1] > baselines['lower_freq']):
            d[k][2] = 'evolving_concert-goer'
        elif (v[0] > baselines['avg_price']) & (v[1] < baselines['lower_freq']):
            d[k][2] = 'high-value_prospect'
        elif (v[0] < baselines['avg_price']) & (v[1] < baselines['lower_freq']):
            d[k][2] = 'one-timer'
        else:
            raise Exception(f"Unable to segment. Check customer {k}: {v}, {baselines}")
    
    return d

In [355]:
clx_segments = segmentation(tix_df, 'clx', clx_baselines)
pop_segments = segmentation(tix_df, 'pops', pop_baselines)

[27.46764705882353, 49, 'committed_low-budget']

## Combine into a df

In [357]:
customer_segments = {int(k): {'recency': v, 
                              'clx_segment': None, 
                              'pops_segment': None} for k,v in cust_recency.items()}

In [358]:
for k,v in clx_segments.items():
    customer_segments[int(k)]['clx_segment'] = v[2]
    
for k,v in pop_segments.items():
    customer_segments[int(k)]['pops_segment'] = v[2]

In [363]:
customer_segment_df = pd.DataFrame(customer_segments).transpose().reset_index()

## Write to excel file

In [366]:
with pd.ExcelWriter('customer_segments.xlsx') as writer:
    pd.DataFrame(customer_segment_df).to_excel(writer,
                                             engine='xlsxwriter', 
                                             index=False)